In [ ]:
#data manip
import pandas as pd
import numpy as np

#visual
import matplotlib.pyplot as plt
import seaborn as sns

#navigating files
import pathlib
from pathlib import Path
import os
import shutil

#reading json
import json

#spectrograms
import librosa, librosa.display

#progress bar
import tqdm
from tqdm import tqdm

In [54]:
#function to remove .DS_Store on MACOS
def remove_DS_Store(dir_path="./final_data/"):
    l = []
    for path in Path(dir_path).rglob(".DS_Store"):
        #assert (os.path.isfile(path))
        #l.append(path)
        #print(path.name)
        path.unlink()
    #assert (len(l) == 0)
    #print(f"There are {len(l)} files with name .DS_Store")

#running func
remove_DS_Store(dir_path="./Coswara-Data")

In [55]:
remove_DS_Store(dir_path="./Coswara-Data/cleaned_data/")

In [9]:
#create dataframe and append values to it 
def gen_dataframe(fols_path="./final_data"):
    
    #creating empty lists 
    ages = []
    cov_stat = []
    genders = []
    wav_paths = []
    
    #listing files
    fols = os.listdir("./final_data")
    fols.remove("combined_data.csv")
    
    #empty dataframe
    df = pd.DataFrame(columns=["ID","Age","Covid_status","Gender", "wav_path"])

    for fol in fols:
        full_wav_path = f"./final_data/{fol}/cough-heavy.wav"
        wav_paths.append(full_wav_path)
        with open(f"./final_data/{fol}/metadata.json") as json_file:
            data = json.load(json_file)
            ages.append(data['a'])
            cov_stat.append(data['covid_status'])
            genders.append(data['g'])
    df['ID'] = fols
    df['Age'] = ages
    df['Covid_status'] = cov_stat
    df['Gender'] = genders
    df['wav_path'] = wav_paths
    
    maps = {
            "healthy":"Negative",
            "no_resp_illness_exposed":"Negative",
            "positive_moderate":"Positive",
            "positive_asymp":"Positive",
            "positive_mild":"Positive",
            "recovered_full":"Negative",
            "resp_illness_not_identified":"Unknown"
           }

    df['Covid_status'] = df['Covid_status'].map(maps)
    df['nums'] = list(range(df.shape[0]))
    df['new_name'] = "./final_data/" + df['ID'] + "/" + df['nums'].astype(str)+ "_" + df['Covid_status']+ "_" + df['Gender']+ "_" + df['Age'].astype(str) + ".wav"
    df.drop(['ID'],axis=1,inplace=True)
    df.drop(['nums'],axis=1,inplace=True)
    df.drop(['wav_path'],axis=1,inplace=True)
    df.drop(['new_name'],axis=1,inplace=True)
    
    
    return df 

In [10]:
df = gen_dataframe(fols_path="./final_data")

In [78]:
def rename_move_files():
    #renaming the files 
    old_names = list(df['wav_path'])
    new_names = list(df['new_name'])
    for idx, e in enumerate(old_names):
        os.rename(e, new_names[idx])
    results = ['Positive', 'Negative', 'Unknown']
    for res in results: 
            pathlib.Path(f"./cleaned_data/{res}").mkdir(parents=True, exist_ok=True)
            for files in Path("./final_data/").rglob("*.wav"):
                if res in files.parts[2]: 
                    shutil.move(files, f"./cleaned_data/{res}/{files.parts[2]}")

In [47]:
def spectrogram_from_wav():
    results = ['Positive', 'Negative', 'Unknown']
    for res in results: 
        pathlib.Path(f"spectrograms/{res}").mkdir(parents=True, exist_ok=True)
        for files in tqdm(os.listdir(f"./cleaned_data/{res}")):
            filename = f"cleaned_data/{res}/{files}"
            #print(filename)
            x, sr = librosa.load(filename, mono=True)
            plt.specgram(x, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap='inferno', sides='default', mode='default', scale='dB');
            plt.axis('off');
            plt.savefig(f"./spectrograms/{res}/{files[:-4]}.png")
            plt.clf()

In [49]:
spectrogram_from_wav()

100%|██████████| 73/73 [00:34<00:00,  2.09it/s]


<Figure size 432x288 with 0 Axes>

In [57]:
spectograms = "./spectrograms"

In [59]:
#usefule module to split folders into train-val
import splitfolders
splitfolders.ratio(spectograms, output="./sepparated_specs", 
                   seed=1337, ratio=(0.8,0.2))

Copying files: 1287 files [00:03, 402.77 files/s]


In [ ]:
#Thank you SO for this fine-tune over plot font sizes!!!
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title